In [1]:
import process as p
import pandas as pd
import dmc

In [2]:
df = p.processed_data()
df = p.shuffle(df)

In [3]:
start, end, split = pd.Timestamp('2014-1-1'), pd.Timestamp('2014-12-31'), pd.Timestamp('2014-10-1')
df.orderDate = pd.to_datetime(df.orderDate)
mask = (df.orderDate >= start) & (df.orderDate <= end)
df_full = df[mask]
te_size = 5000
tr_size = 5000
X, Y, fts = dmc.transformation.transform_preserving_header(df_full, scaler=dmc.transformation.scale_features, binary_target=True)
train = X[:tr_size], Y[:tr_size]
test = X[tr_size:tr_size+te_size], Y[tr_size:tr_size+te_size]

/home/bpm1/BigData/run-dmc/venv/lib/python3.4/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
#res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)

In [5]:
'''Calculate impact of features for n iterations'''
impact_map = dict()
feature_set = set(fts)
for feature in fts:
        impact_map[feature] = 0.
for i in range(6):
    print(i)
    res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)
    for feature in feature_set:
            impact_map[feature] = impact_map[feature] + res_tree.decrement[feature]
import operator
sorted_impact_list = sorted(impact_map.items(), key=operator.itemgetter(1))
print(sorted_impact_list)


0
1
2
3
4
5
[('t_order_cheapestPrice', -0.066199999999999926), ('t_customer_voucherCount', -0.061999999999999944), ('t_article_boughtCountGlobal', -0.060799999999999965), ('orderWeek', -0.058999999999999941), ('t_unisizeOffset', -0.055199999999999916), ('t_order_meanPrice', -0.054599999999999871), ('rrp', -0.053399999999999892), ('t_order_priceStd_A', -0.046799999999999953), ('colorReturnProb', -0.046599999999999753), ('t_order_article_sameArticlesCount_DiffColor', -0.044199999999999795), ('t_customer_boughtArticleCount', -0.043399999999999883), ('orderWeekDay', -0.040599999999999858), ('t_sizeCodeNumerized', -0.039000000000000035), ('t_wsv', -0.036799999999999833), ('orderMonth', -0.03620000000000001), ('t_voucher_isValueVoucher', -0.035599999999999965), ('binnedColorCode', -0.034599999999999964), ('productGroup', -0.032400000000000095), ('t_order_boughtArticleCount', -0.032399999999999873), ('t_voucher_is10PercentVoucher', -0.032399999999999651), ('totalOrderShare', -0.03179999999999

In [6]:
res_tree[(res_tree.decrement < 0)]

,decrement,precision
t_article_boughtCountGlobal,-0.0112,0.6804
t_order_hasVoucher_A,-0.0050,0.6742
t_unisize,-0.0038,0.6730
t_order_duplicateCount,-0.0042,0.6734
t_order_totalPrice,-0.0046,0.6738
t_customer_voucherCount,-0.0108,0.6800
t_article_availableSizes,-0.0028,0.6720
orderWeekDay,-0.0114,0.6806
orderDayOfYear,-0.0016,0.6708
productGroupReturnProb,-0.0044,0.6736


In [7]:
kick_features = 20
for i in range(kick_features):
    res_tree = dmc.evaluation.evaluate_without_one_feature(train[0], train[1], test[0], test[1], fts, sorted_impact_list[i][0], dmc.classifiers.DecisionTree, 10)
    print(res_tree)

TypeError: evaluate_without_one_feature() takes 7 positional arguments but 8 were given